[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/reznikmm/ada-howto/advent-2020?filepath=%2Fhome%2Fjovyan%2Fnb%2F14%2F14.ipynb)

# Day 14

Firstly, we need some standard packages to read the text file and keep a program state.

In [1]:
with Ada.Text_IO;
with Ada.Strings.Fixed;
with Ada.Strings.Maps;
with Ada.Strings.Maps.Constants;
with Ada.Containers.Hashed_Maps;

To represent 36 bit words we use a modular type.

In [2]:
type Word is mod 2 ** 36;

package Word_IO is new Ada.Text_IO.Modular_IO (Word);

For mask let's use a record type of two words. The first work keeps ordinary bits and `0` instead of `X`. The second words keep `0` for `0` or `1` and `1` for `X`. We will use two character mappings to translate mask text image into `Word`.

In [3]:
type Address_Bits is record
   Plain_Bits    : Word;  --  Mask where X replaced by 0
   Floating_Bits : Word;  --  Mask where X replaced by 1 and others are 0
end record;

Drop_X_Map : constant Ada.Strings.Maps.Character_Mapping :=
  Ada.Strings.Maps.To_Mapping ("X", "0");

X_To_1_Map : constant Ada.Strings.Maps.Character_Mapping :=
  Ada.Strings.Maps.To_Mapping ("X10", "100");

Declare a dedicated function to cast a string of `0` and `1` into a `Word`. Ada understantd binary images in form of `2#10...10#`.

In [4]:
function Binary_Value (Text : String) return Word;

function Binary_Value (Text : String) return Word is
   Image : constant String := "2#" & Text & "#";
   Result : Word;
   Last : Natural;
begin
   Word_IO.Get
     (From => Image,
      Item => Result,
      Last => Last);
   pragma Assert (Last = Image'Last);
   return Result;
end Binary_Value;

To replresent the memory we use a hash (Word to Word) map.

In [5]:
function Hash (Value : Word) return Ada.Containers.Hash_Type is
  (Ada.Containers.Hash_Type'Mod (Value));

package Word_Maps is new Ada.Containers.Hashed_Maps
  (Key_Type        => Word,
   Element_Type    => Word,
   Hash            => Hash,
   Equivalent_Keys => "=");

Memory : Word_Maps.Map;

Procedure `Assign` instatiates floating bits into plain bits and modifies `Memory`.

In [6]:
procedure Assign (Address : Address_Bits; Value : Word);
--  Assign Value to Memory at given Address with all combinations of floating bits.

procedure Assign (Address : Address_Bits; Value : Word) is
begin
   if Address.Floating_Bits = 0 then
      Memory.Include (Address.Plain_Bits, Value);
   else
      declare
         Bit : Word := 1;
      begin
         for J in 1 .. 36 loop
            if (Address.Floating_Bits and Bit) /= 0 then
               Assign
                 ((Address.Plain_Bits, Address.Floating_Bits xor Bit),
                  Value);
               Assign
                 ((Address.Plain_Bits xor Bit,
                   Address.Floating_Bits xor Bit),
                  Value);
               exit;
            end if;

            Bit := Bit * 2;
         end loop;
      end;
   end if;
end Assign;

Define current mask and input file objects.

In [7]:
Mask  : Address_Bits := (0, 0);
Input : Ada.Text_IO.File_Type;

Now we are ready to do the actual job

In [8]:
Ada.Text_IO.Open (Input, Ada.Text_IO.In_File, "/home/jovyan/md/14/input");
   
while not Ada.Text_IO.End_Of_File (Input) loop
   declare
      Line : String := Ada.Text_IO.Get_Line (Input);
   begin
      if Line (1 .. 7) = "mask = " then
         Mask.Plain_Bits := Binary_Value
           (Ada.Strings.Fixed.Translate
              (Line (8 .. Line'Last), Drop_X_Map));
         Mask.Floating_Bits := Binary_Value
           (Ada.Strings.Fixed.Translate
              (Line (8 .. Line'Last), X_To_1_Map));
      elsif Line (1 .. 4) = "mem[" then
         declare
            First, Last : Positive;
            Address     : Word;
            Argument    : Word;
         begin
            Ada.Strings.Fixed.Find_Token
              (Source => Line,
               Set    => Ada.Strings.Maps.Constants.Decimal_Digit_Set,
               Test   => Ada.Strings.Inside,
               First  => First,
               Last   => Last);
            Word_IO.Get
              (Line (First .. Last),
               Address,
               Last => First);
            pragma Assert (First = Last);

            Ada.Strings.Fixed.Find_Token
              (Source => Line,
               From   => Last + 1,
               Set    => Ada.Strings.Maps.Constants.Decimal_Digit_Set,
               Test   => Ada.Strings.Inside,
               First  => First,
               Last   => Last);
            Word_IO.Get
              (Line (First .. Last),
               Argument,
               Last => First);
            pragma Assert (First = Last);

            Address := Address or Mask.Plain_Bits;
            Assign ((Address, Mask.Floating_Bits), Argument);
         end;
      end if;
   end;
end loop;

Now we have memory filled with values. Let's sum all of them.

In [9]:
declare
   Result : Long_Integer := 0;
begin
   for Value of Memory loop
      Result := Result + Long_Integer (Value);
   end loop;
   Ada.Text_IO.Put_Line (Result'Image);
end;

 2881082759597


----
[Back to Table of Contents](https://github.com/reznikmm/ada-howto/tree/advent-2020)